In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader


In [2]:

from torchvision.transforms import v2



In [3]:

training_data = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=v2.Compose([
        v2.ToTensor(),
        v2.RandomHorizontalFlip(),
        v2.RandomResizedCrop(size=32, scale=(0.85, 1.0), antialias=False),
        v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    ])
)
test_data = torchvision.datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=v2.Compose([
        v2.ToTensor(),
    ])
)

# split the training data into training and validation
training_data, validation_data = torch.utils.data.random_split(training_data, [.8, .2], generator=torch.Generator().manual_seed(42))
batch_size = 128

# Create the dataloaders

train_dataloader = DataLoader(training_data,
                              batch_size=batch_size,
                              pin_memory=True,
                              shuffle=True,
                              num_workers=4)
validation_dataloader = DataLoader(validation_data,
                                   batch_size=batch_size,
                                   pin_memory=True,  # copy the data to the GPU
                                   shuffle=False,
                                   num_workers=4)  # number of subprocesses to use for data loading




/home/adrianhurtado/miniforge3/envs/ai_train/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def preprocess(X, Y):
    # CIFAR-10 is *color*
    return X.view(-1, 3, 32, 32).to(device), Y.to(device)


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        for b in self.dl:
            yield (self.func(*b))


train_dataloader = WrappedDataLoader(train_dataloader, preprocess)
validation_dataloader = WrappedDataLoader(validation_dataloader, preprocess)

In [6]:
from torch import nn

# Lets try a downsampler with convol
class Downsampler(nn.Module):
    def __init__(self, in_channels, out_channels):

        kernel_size=3
        stride=1
        padding=1 # keep the same size at the end
        super(Downsampler, self).__init__()

        layers=[
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels,out_channels,kernel_size,stride,padding),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.MaxPool2d(2)]
        self.downsample=nn.Sequential(*layers)

    def forward(self, inputs):
        return self.downsample(inputs)

class ConvNextBlock(nn.Module):
    """This block of operations is loosely based on this paper:
    start with an input with a 7 by 7 kernel, however with same padding it will not change the size of the image.
    Then we apply a normalization layer, and follow with 4 filters, aka patches,per channel.
    
    """

    def __init__(self, in_channels, shape):
        super(ConvNextBlock, self).__init__()
        # Depthwise, seperable convolution with a large number of output filters:
        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=in_channels,
                               groups=in_channels,# this is what makes it seperable? in_channel number of groups
                               kernel_size=[7, 7],
                               padding='same')# keep the same size
        #This is like an expand, 

        self.norm = nn.LayerNorm([in_channels, *shape])
        # Two more convolutions:
        self.conv2 = nn.Conv2d(in_channels=in_channels,
                               out_channels=4 * in_channels,
                               kernel_size=1)
        # Then we do the unrolling of the patches, and apply a non-linear activation function.
        self.conv3 = nn.Conv2d(in_channels=4 * in_channels,
                               out_channels=in_channels,
                               kernel_size=1
                               )

    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.norm(x)
        x = self.conv2(x)
        x = torch.nn.functional.gelu(x)
        x = self.conv3(x)
        # This makes it a residual network:
        return x + inputs

class Classifier(nn.Module):

    def __init__(self, n_initial_filters, n_stages, blocks_per_stage):
        super(Classifier, self).__init__()
        # our first example is set with 64,4,2 meaning 64 initial filters, 4 stages, 2 blocks per stage
        # This is a downsampling convolution that will produce patches of output.
        # This is similar to what vision transformers do to tokenize the images.
        self.stem = nn.Conv2d(in_channels=3,
                              out_channels=n_initial_filters,
                              kernel_size=1,
                              stride=1)
        # Here is the initial convolution which takes our 32 x 32 image with 3 channels and 
        # converts it to 64 channels with a 1x1 convolution

        # In other words, this stem is a patching operation that converts the input image into a set of patches.
        # 3 to 64.  maybe here it makes sense to make n_intial_filters = 3 * filter_per_channel.. say 32

        current_shape = [32, 32]
        self.norm1 = nn.LayerNorm([n_initial_filters, *current_shape])
        # self.norm1 = WrappedLayerNorm()
        current_n_filters = n_initial_filters

        self.layers = nn.Sequential()
        for i, n_blocks in enumerate(range(n_stages)):
            # Add a convnext block series:
            for _ in range(blocks_per_stage):
                self.layers.append(ConvNextBlock(in_channels=current_n_filters, shape=current_shape))
            # Add a downsampling layer:
            if i != n_stages - 1:
                # Skip downsampling if it's the last layer!
                self.layers.append(Downsampler(
                    in_channels=current_n_filters,
                    out_channels=2 * current_n_filters,
                )
                )
                # Double the number of filters:
                current_n_filters = 2 * current_n_filters
                # Cut the shape in half:
                current_shape = [cs // 2 for cs in current_shape]
                # here w eare doing a downsampling operation, which is a patching operation.
                # We record the current shape

        self.head = nn.Sequential(
            nn.Flatten(),
            nn.LayerNorm(current_n_filters),
            nn.Linear(current_n_filters, 10)
        )
        # self.norm2 = nn.InstanceNorm2d(current_n_filters)
        # # This brings it down to one channel / class
        # self.bottleneck = nn.Conv2d(in_channels=current_n_filters, out_channels=10, 
        #                                   kernel_size=1, stride=1)

    def forward(self, inputs):

        x = self.stem(inputs)
        # Apply a normalization after the initial patching:
        x = self.norm1(x)

        # Apply the main chunk of the network:
        x = self.layers(x)

        # Normalize and readout:
        x = nn.functional.avg_pool2d(x, x.shape[2:])
        x = self.head(x)

        return x

In [7]:
#model = Classifier(64, 4, 2)
#model.cuda()
# The first test will be to set the initial filter to a multiple of . Actually, Why don't I just convert the image to a 1 channel if color should not matter for these images.  I can then normalize the image as well.  
# 3 stages 
# 2 blocks per stage
model = Classifier(64, 3, 3)
model.cuda()


Classifier(
  (stem): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  (norm1): LayerNorm((64, 32, 32), eps=1e-05, elementwise_affine=True)
  (layers): Sequential(
    (0): ConvNextBlock(
      (conv1): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=same, groups=64)
      (norm): LayerNorm((64, 32, 32), eps=1e-05, elementwise_affine=True)
      (conv2): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (conv3): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): ConvNextBlock(
      (conv1): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=same, groups=64)
      (norm): LayerNorm((64, 32, 32), eps=1e-05, elementwise_affine=True)
      (conv2): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (conv3): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (2): ConvNextBlock(
      (conv1): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=same, groups=64)
      (norm): LayerNorm((64, 32, 32), eps=1e-05, elementwise_affine=

In [8]:
def evaluate(dataloader, model, loss_fn, val_bar):
    # Set the model to evaluation mode 
    model.eval()
    size = len(dataloader)
    num_batches = len(dataloader)
    loss, correct = 0, 0

    # We can save computation and memory by not calcualting the gradients
    with torch.no_grad():
        for X, Y in dataloader:
            pred = model(X)
            loss += loss_fn(pred, Y).item()
            correct += torch.sum(torch.argmax(pred, dim=1) == Y).item()
            val_bar.update()
    loss /= num_batches
    correct /= (size * batch_size)
    accuracy = 100 * correct
    return accuracy, loss

In [9]:
def train_one_epoch(dataloader, model, loss_fn, optimizer, progress_bar):
    # Set the model to training mode
    model.train()
    for batch, (X, Y) in enumerate(dataloader):
        pred = model(X)
        l = loss_fn(pred, Y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        progress_bar.update()


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

In [11]:
from tqdm import tqdm

In [12]:
epochs = 30
for j in range(epochs):
    with tqdm(total=len(train_dataloader), position=0, leave=True, desc=f"Train Epoch {j}") as train_bar:
        train_one_epoch(train_dataloader, model, loss_fn, optimizer, train_bar)

    # checking on the training loss and accuracy once per epoch

    with tqdm(total=len(train_dataloader), position=0, leave=True, desc=f"Validate (train) Epoch {j}") as train_eval:
        acc, loss = evaluate(train_dataloader, model, loss_fn, train_eval)

        print(f"Epoch {j}: training loss: {loss:.3f}, accuracy: {acc:.3f}")
    with tqdm(total=len(validation_dataloader), position=0, leave=True, desc=f"Validate Epoch {j}") as val_bar:
        acc_val, loss_val = evaluate(validation_dataloader, model, loss_fn, val_bar)
        print(f"Epoch {j}: validation loss: {loss_val:.3f}, accuracy: {acc_val:.3f}")

Validate (train) Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:34<00:00,  9.02it/s]


Epoch 0: training loss: 1.184, accuracy: 57.640


Validate Epoch 0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:08<00:00,  8.94it/s]


Epoch 0: validation loss: 1.198, accuracy: 56.764


Validate (train) Epoch 1: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:33<00:00,  9.41it/s]


...

Epoch 28: validation loss: 0.481, accuracy: 86.175


Validate (train) Epoch 29: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:34<00:00,  8.96it/s]


Epoch 29: training loss: 0.053, accuracy: 98.033


Validate Epoch 29: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:08<00:00,  8.88it/s]

Epoch 29: validation loss: 0.465, accuracy: 87.134


In [14]:
!nvidia-smi


Thu Feb 22 17:12:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   84C    P0              27W /  45W |   3246MiB /  4096MiB |     52%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

For this homework I made one main modification which was to change my downsampling strategy to using 2D max pooling.  The downsampling block contains 2 convolution + Gelu layers which maintain the kernel size until the last pass which does the max pooling.  The orinal downsampler was a bit similar using only the convolution as the downsampler using a stride of 2.  I'm not sure what will be better but I figured I'd can give this a try since pooling was introduce in the lecture.  Additionally I set my Classifier with 64 intial filter 3 stages and 3 blocks per stage which Reduces the amount of downsampling but widens each layer compared to the original 4 stages with 2 blocks per stage.  

## Note

In a previous version of a notebook I trained a model with the same 64,3,3 settings with the original downsampler and after 30 epochs I got 96% training accuracy and 82% validation accuracy.  In this case I get 87% training accuracy so I do see an improvement.  